## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [ ]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "dev" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "13.219.80.155" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [5]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://13.219.80.155:5000'


In [6]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-ruoke/1', creation_time=1751362233639, experiment_id='1', last_update_time=1751362233639, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-ruoke/0', creation_time=1751360770765, experiment_id='0', last_update_time=1751360770765, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/07/01 17:34:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/01 17:34:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://mlflow-artifacts-remote-ruoke/1/5b11b0eafe974f37b6940cee7979726e/artifacts'
🏃 View run carefree-fox-373 at: http://13.219.80.155:5000/#/experiments/1/runs/5b11b0eafe974f37b6940cee7979726e
🧪 View experiment at: http://13.219.80.155:5000/#/experiments/1


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-ruoke/1', creation_time=1751362233639, experiment_id='1', last_update_time=1751362233639, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-ruoke/0', creation_time=1751360770765, experiment_id='0', last_update_time=1751360770765, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [13]:
from mlflow.tracking import MlflowClient
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [17]:

run_id='f5b4bd2300ec4943955d253603ec3e29'
model_uri = f"runs:/{run_id}/models"
print(f"Using model URI: {model_uri}")
#register model
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-xgboost")


Using model URI: runs:/f5b4bd2300ec4943955d253603ec3e29/models


Registered model 'nyc-taxi-xgboost' already exists. Creating a new version of this model...


RestException: RESOURCE_DOES_NOT_EXIST: Run with id=f5b4bd2300ec4943955d253603ec3e29 not found